In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
temp_df = pd.read_csv('/content/drive/MyDrive/NLP/IMDB Dataset.csv')

In [ ]:
df = temp_df.iloc[:10000]

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
# drop the duplicate values
df.drop_duplicates(inplace=True)

<ipython-input-90-9b8da17d5385>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [ ]:
# remove HTML Tags
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [ ]:
df['review'] = df['review'].apply(remove_tags)

<ipython-input-92-00e93c2a3043>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [ ]:
df['review'] = df['review'].apply(lambda x:x.lower())

<ipython-input-93-afca29351ee0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-94-977c865be4a1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [ ]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [ ]:
import gensim

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
import nltk
nltk.download('punkt')

story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    
    for sent in raw_sent:
        story.append(simple_preprocess(sent))  #Convert a document into a list of lowercase tokens
    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
model = gensim.models.Word2Vec(window=7,min_count=2)  

# size: The number of dimensions of the embeddings and the default is 100.
# window: The maximum distance between a target word and words around the target word. The default window is 5.
# min_count: The minimum count of words to consider when training the model; 
# words with occurrence less than this count will be ignored. The default for min_count is 5.

In [ ]:
model.build_vocab(story)

In [ ]:
model.corpus_count

In [ ]:
model.epochs

5

In [ ]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5684256, 6212140)

In [ ]:
# length of vocabulary
len(model.wv.index_to_key)

18211

In [ ]:
model.wv.index_to_key

In [ ]:
model.wv.similar_by_word('movie')

[('film', 0.8934369683265686),
 ('documentary', 0.7885916829109192),
 ('sequel', 0.7532411813735962),
 ('entertainment', 0.7009662389755249),
 ('show', 0.6931667327880859),
 ('viewing', 0.6894593834877014),
 ('flick', 0.6889171004295349),
 ('picture', 0.6826016902923584),
 ('experience', 0.6809185743331909),
 ('so', 0.6716386079788208)]

In [ ]:
# Apply avg word2vec
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [ ]:
document_vector(df['review'].values[0])

array([-0.17416987,  0.3061478 ,  0.12980434,  0.1768537 , -0.02826198,
       -0.5146865 ,  0.09502754,  0.69622225, -0.30391634, -0.14518598,
       -0.18320498, -0.42665398, -0.01299498,  0.22940291,  0.15633123,
       -0.28116152,  0.08780271, -0.22118591, -0.04905951, -0.52488166,
        0.15381692,  0.06376027,  0.12289793, -0.21559939, -0.11366997,
       -0.20107697, -0.29660064, -0.00217378, -0.20624891,  0.05078185,
        0.28722304, -0.00236025,  0.15103771, -0.2505364 , -0.03590529,
        0.33698732,  0.15012881, -0.33560959, -0.14470468, -0.6064261 ,
        0.04470306, -0.238573  , -0.12372695, -0.02153799,  0.42672688,
       -0.11321954, -0.16910443,  0.01493817,  0.02441238,  0.2573005 ,
        0.15919037, -0.3128441 , -0.2627871 , -0.03420112, -0.09080354,
        0.20948257,  0.173369  ,  0.07943306, -0.19160372,  0.10338593,
        0.00191102,  0.14696969, -0.04690636, -0.06037341, -0.24776855,
        0.18611817,  0.14766537,  0.13533916, -0.41941258,  0.36

In [ ]:
from tqdm import tqdm   # To get the progress bar

In [ ]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [03:56<00:00, 42.17it/s]


In [ ]:
X = np.array(X)

In [ ]:
X[0]

array([-0.17416987,  0.3061478 ,  0.12980434,  0.1768537 , -0.02826198,
       -0.5146865 ,  0.09502754,  0.69622225, -0.30391634, -0.14518598,
       -0.18320498, -0.42665398, -0.01299498,  0.22940291,  0.15633123,
       -0.28116152,  0.08780271, -0.22118591, -0.04905951, -0.52488166,
        0.15381692,  0.06376027,  0.12289793, -0.21559939, -0.11366997,
       -0.20107697, -0.29660064, -0.00217378, -0.20624891,  0.05078185,
        0.28722304, -0.00236025,  0.15103771, -0.2505364 , -0.03590529,
        0.33698732,  0.15012881, -0.33560959, -0.14470468, -0.6064261 ,
        0.04470306, -0.238573  , -0.12372695, -0.02153799,  0.42672688,
       -0.11321954, -0.16910443,  0.01493817,  0.02441238,  0.2573005 ,
        0.15919037, -0.3128441 , -0.2627871 , -0.03420112, -0.09080354,
        0.20948257,  0.173369  ,  0.07943306, -0.19160372,  0.10338593,
        0.00191102,  0.14696969, -0.04690636, -0.06037341, -0.24776855,
        0.18611817,  0.14766537,  0.13533916, -0.41941258,  0.36

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [ ]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7611417125688533